First, import needed modules

In [46]:
import numpy as np
import pandas as pd
import os

Initalize variables

In [47]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [48]:
btw02_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW02_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG', sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw05_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW05_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw09_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW09_Erststimmen_Wbz.csv', sep=';', encoding = 'UTF-8-SIG',skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw13_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW13_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw17_es = pd.read_csv(f'{path}/data/elections/btw/es/btw17_wbz_erststimmen.csv', sep=';', encoding = 'Windows-1252', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})

btw02_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW02_Zweitstimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG', sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw05_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW05_Zweitstimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw09_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW09_Zweitstimmen_Wbz.csv', sep=';', encoding = 'UTF-8-SIG',skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw13_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW13_Zweitstimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw17_zs = pd.read_csv(f'{path}/data/elections/btw/zs/btw17_wbz_zweitstimmen.csv', sep=';', encoding = 'Windows-1252', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})

/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_28939/2269252993.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  btw05_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW05_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_28939/2269252993.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  btw09_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW09_Erststimmen_Wbz.csv', sep=';', encoding = 'UTF-8-SIG',skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_28939/2269252993.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  btw13_es = pd.read_c

View the data

In [49]:
btw09_zs.head()

,Wahlkreis,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Wahlbezirk,Bezirksart,Wahlberechtigte (A),...,ADM,CM,DKP,DVU,DIE VIOLETTEN,FWD,ödp,PIRATEN,RRP,RENTNER
0,1,01,0,01,0,000,0,000001,0,1416,...,0,0,0,1,0,0,0,23,0,10
1,1,01,0,01,0,000,0,000002,0,1406,...,0,0,0,1,0,0,0,16,0,14
2,1,01,0,01,0,000,0,000003,0,1321,...,0,0,0,0,0,0,0,15,0,12
3,1,01,0,01,0,000,0,000004,0,1405,...,0,0,0,0,0,0,0,19,0,18
4,1,01,0,01,0,000,0,000005,0,1227,...,0,0,0,0,0,0,0,24,0,9


Add year column to dataframes

In [50]:
dfs = [btw02_es, btw05_es, btw09_es, btw13_es, btw17_es, btw02_zs, btw05_zs, btw09_zs, btw13_zs, btw17_zs]
years = [2002, 2005, 2009, 2013, 2017, 2002, 2005, 2009, 2013, 2017]
for df, year in zip(dfs, years):
    df.insert(loc=0, column='year', value=year)

In [51]:
btw17_es.head()

,year,Wahlkreis,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Wahlbezirk,Bezirksart,...,Bündnis C,DIE EINHEIT,DIE VIOLETTEN,FAMILIE,DIE FRAUEN,MIETERPARTEI,Neue Liberale,UNABHÄNGIGE,WGr/EB,Ungekürzte Wahlbezirksbezeichnung
0,2017,1,01,0,01,0,000,0,000001,0,...,0,0,0,0,0,0,0,0,7,–
1,2017,1,01,0,01,0,000,0,000002,0,...,0,0,0,0,0,0,0,0,6,–
2,2017,1,01,0,01,0,000,0,000003,0,...,0,0,0,0,0,0,0,0,1,–
3,2017,1,01,0,01,0,000,0,000004,0,...,0,0,0,0,0,0,0,0,5,–
4,2017,1,01,0,01,0,000,0,000005,0,...,0,0,0,0,0,0,0,0,4,–


Append dataframes

In [52]:
dfs = [btw02_es, btw05_es, btw09_es, btw13_es, btw17_es]
btw_es = pd.concat(dfs, sort=False)
dfs = [btw02_zs, btw05_zs, btw09_zs, btw13_zs, btw17_zs]
btw_zs = pd.concat(dfs, sort=False)
btw_zs.head()

,year,Wahlkreis,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Wahlbezirk,Bezirksart,Wahlberechtigte (A),...,DiB,DM,Die Grauen,du.,MG,MENSCHLICHE WELT,Die Humanisten,Gesundheitsforschung,V-Partei³,Ungekürzte Wahlbezirksbezeichnung
0,2002,1,01,0,01,0,000,1,0,1463,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2002,1,01,0,01,0,000,2,0,1245,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2002,1,01,0,01,0,000,3,0,1399,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2002,1,01,0,01,0,000,4,0,1031,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2002,1,01,0,01,0,000,5,0,1023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Create the AGS, it follows the scheme ll r kk ggg (Land, Regierunsbezirk, Kreis, Gemeinde) -> combine these columns

In [53]:
btw_es.insert(loc=0, column='AGS', value=0)
btw_es['AGS'] = btw_es['Land'] + btw_es['Regierungsbezirk'] + btw_es['Kreis'] + btw_es['Gemeinde']
btw_zs.insert(loc=0, column='AGS', value=0)
btw_zs['AGS'] = btw_zs['Land'] + btw_zs['Regierungsbezirk'] + btw_zs['Kreis'] + btw_zs['Gemeinde']
btw_zs.head()

,AGS,year,Wahlkreis,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Wahlbezirk,Bezirksart,...,DiB,DM,Die Grauen,du.,MG,MENSCHLICHE WELT,Die Humanisten,Gesundheitsforschung,V-Partei³,Ungekürzte Wahlbezirksbezeichnung
0,01001000,2002,1,01,0,01,0,000,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01001000,2002,1,01,0,01,0,000,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01001000,2002,1,01,0,01,0,000,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01001000,2002,1,01,0,01,0,000,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01001000,2002,1,01,0,01,0,000,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Collapse data along AGS column, sums votes over all constituencies in a municipality

In [54]:
btw_es = btw_es.groupby(['AGS', 'year', 'Land'], as_index=False).sum()
btw_zs = btw_zs.groupby(['AGS', 'year', 'Land'], as_index=False).sum()
btw_zs.head()

,AGS,year,Land,Wahlkreis,Verbandsgemeinde,Bezirksart,Wahlberechtigte (A),Wahlberechtigte ohne Sperrvermerk (A1),Wahlberechtigte mit Sperrvermerk (A2),Wahlberechtigte nach § 25 Abs. 2 BWO (A3),...,BGE,DiB,DM,Die Grauen,du.,MG,MENSCHLICHE WELT,Die Humanisten,Gesundheitsforschung,V-Partei³
0,01001000,2002,01,53,0,35,65740,57788,7949,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01001000,2005,01,53,0,35,66970,58379,8591,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01001000,2009,01,58,0,60,69075,60983,8092,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01001000,2013,01,56,0,60,70321,60755,9566,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01001000,2017,01,56,0,60,70105,57929,12176,0.0,...,422.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Add indicator for Erst- und Zweitstimme

In [55]:
dfs = [btw_es, btw_zs]
eszs = ['Erststimme', 'Zweitstimme']
for df, stimme in zip(dfs, eszs):
    df.insert(loc=2, column=stimme, value=1)

In [56]:
btw_zs.head()

,AGS,year,Zweitstimme,Land,Wahlkreis,Verbandsgemeinde,Bezirksart,Wahlberechtigte (A),Wahlberechtigte ohne Sperrvermerk (A1),Wahlberechtigte mit Sperrvermerk (A2),...,BGE,DiB,DM,Die Grauen,du.,MG,MENSCHLICHE WELT,Die Humanisten,Gesundheitsforschung,V-Partei³
0,01001000,2002,1,01,53,0,35,65740,57788,7949,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01001000,2005,1,01,53,0,35,66970,58379,8591,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01001000,2009,1,01,58,0,60,69075,60983,8092,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01001000,2013,1,01,56,0,60,70321,60755,9566,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01001000,2017,1,01,56,0,60,70105,57929,12176,...,422.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


ES: Sum votes for other, Union and Die Linke parties

In [57]:
# list(btw_es.columns)

In [58]:
andere = ['REP',
'GRAUE',
'TIERSCHUTZ',
'NPD',
'ÖDP',
'PBC',
'FRAUEN',
'BP',
'FAMILIE',
'CM',
'BÜSO',
'HP',
'VIOLETTEN',
'AUFBRUCH',
'ZENTRUM',
'KPD',
'PRG',
'SCHILL',
'DEUTSCHLAND',
'DKP',
'DSU',
'FP D',
'ÜBRIGE',
'Offensive D',
'Die Tierschutzpartei',
'DIE FRAUEN',
'BüSo',
'Deutschland',
'AGFG',
'APPD',
'MLPD',
'DIE PARTEI',
'HUMANWIRTSCHAFTSPARTEI',
'STATT Partei',
'UNABHÄNGIGE',
'Übrige', 
'Volksabstimmung',
'ADM',
'DIE VIOLETTEN',
'ödp',
'PIRATEN',
'RRP',
'Freie Union',
'Tierschutzpartei',
'Bündnis 21/RRP',
'RENTNER',
'AfD',
'BIG',
'pro Deutschland',
'FREIE WÄHLER',
'PARTEI DER VERNUNFT',
'Die PARTEI',
'Bergpartei',
'BGD',
'NEIN',
'WGr/EB',
'PDV',
'SGP',
'DIE RECHTE',
'Tierschutzallianz',
'B*',
'Die Grauen',
'du',
'MG',
'MENSCHLICHE WELT',
'Gesundheitsforschung',
'V-Partei³',
'Bündnis C',
'DIE EINHEIT',
'MIETERPARTEI',
'Neue Liberale']
btw_es['Andere'] = btw_es[andere].sum(axis=1) # sum votes for 'Other' Parties
union = ['CDU', 'CSU']
btw_es['Union'] = btw_es[union].sum(axis=1) # sum votes for 'Union' Parties
linke = ['Die Linke', 'DIE LINKE', 'PDS']
btw_es['Linke'] = btw_es[linke].sum(axis=1) # sum votes for 'Linke' Parties
invalid = ['Ungültige', 'Ungültig']
btw_es['Ungültig'] = btw_es[invalid].sum(axis=1) # sum invalid votes

ZS: Sum votes for other, Union and DIE LINKE parties

In [59]:
# list(btw_zs.columns)

In [60]:
andere = ['REP',
'GRAUE',
'Tierschutz',
'NPD',
'ödp',
'PBC',
'FRAUEN',
'BP',
'FAMILIE',
'CM',
'BüSo',
'HP',
'Violetten',
'AUFBRUCH',
'ZENTRUM',
'KPD',
'PRG',
'Schill',
'Offensive D',
'Die Tierschutzpartei',
'DIE FRAUEN',
'Deutschland',
'AGFG',
'APPD',
'50plus',
'MLPD',
'Die Partei',
'PSG',
'Pro DM',
'Volksabstimmung',
'ADM',
'DKP',
'DVU',
'DIE VIOLETTEN',
'FWD',
'PIRATEN',
'RRP',
'RENTNER',
'Tierschutzpartei',
'ÖDP',
'Bündnis 21/RRP',
'AfD',
'BIG',
'pro Deutschland',
'DIE RECHTE',
'FREIE WÄHLER',
'Partei der Nichtwähler',
'PARTEI DER VERNUNFT',
'Die PARTEI',
'PDV',
'SGP',
'AD-DEMOKRATEN',
'Tierschutzallianz',
'B*',
'BGE',
'DiB',
'DM',
'Die Grauen',
'du.',
'MG',
'MENSCHLICHE WELT',
'Die Humanisten',
'Gesundheitsforschung',
'V-Partei³']
btw_zs['Andere'] = btw_zs[andere].sum(axis=1) # sum votes for 'Other' Parties
union = ['CDU', 'CSU']
btw_zs['Union'] = btw_zs[union].sum(axis=1) # sum votes for 'Union' Parties
linke = ['Die Linke', 'DIE LINKE', 'PDS']
btw_zs['Linke'] = btw_zs[linke].sum(axis=1) # sum votes for 'Linke' Parties
invalid = ['Ungültige', 'Ungültig']
btw_zs['Ungültig'] = btw_zs[invalid].sum(axis=1) # sum invalid votes

Concatenate Erst- and Zweitstimme datasets

In [61]:
dfs = [btw_es, btw_zs]
btw = pd.concat(dfs, sort=False)
btw = btw.sort_values(['AGS', 'year'])
btw.head(10)

,AGS,year,Erststimme,Land,Wahlkreis,Verbandsgemeinde,Bezirksart,Wahlberechtigte (A),Wahlberechtigte ohne Sperrvermerk (A1),Wahlberechtigte mit Sperrvermerk (A2),...,Pro DM,DVU,FWD,Partei der Nichtwähler,AD-DEMOKRATEN,BGE,DiB,DM,du.,Die Humanisten
0,01001000,2002,1.0,01,53,0,35,65740,57788,7949,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,01001000,2002,NaN,01,53,0,35,65740,57788,7949,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01001000,2005,1.0,01,53,0,35,66970,58379,8591,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01001000,2005,NaN,01,53,0,35,66970,58379,8591,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01001000,2009,1.0,01,58,0,60,69075,60983,8092,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01001000,2009,NaN,01,58,0,60,69075,60983,8092,...,0.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01001000,2013,1.0,01,56,0,60,70321,60755,9566,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01001000,2013,NaN,01,56,0,60,70321,60755,9566,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01001000,2017,1.0,01,56,0,60,70105,57929,12176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01001000,2017,NaN,01,56,0,60,70105,57929,12176,...,0.0,0.0,0.0,0.0,0.0,422.0,0.0,0.0,0.0,0.0


Add post-treatment indicator

In [62]:
years = [2017, 2013]
for year in years:
    btw.insert(loc=3, column=f'post_{year}', value=0)
    btw[f'post_{year}'] = np.where(btw['year'] >= year, 1, 0)
btw.head(10)

,AGS,year,Erststimme,post_2013,post_2017,Land,Wahlkreis,Verbandsgemeinde,Bezirksart,Wahlberechtigte (A),...,Pro DM,DVU,FWD,Partei der Nichtwähler,AD-DEMOKRATEN,BGE,DiB,DM,du.,Die Humanisten
0,01001000,2002,1.0,0,0,01,53,0,35,65740,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,01001000,2002,NaN,0,0,01,53,0,35,65740,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01001000,2005,1.0,0,0,01,53,0,35,66970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01001000,2005,NaN,0,0,01,53,0,35,66970,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01001000,2009,1.0,0,0,01,58,0,60,69075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01001000,2009,NaN,0,0,01,58,0,60,69075,...,0.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01001000,2013,1.0,1,0,01,56,0,60,70321,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01001000,2013,NaN,1,0,01,56,0,60,70321,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01001000,2017,1.0,1,1,01,56,0,60,70105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01001000,2017,NaN,1,1,01,56,0,60,70105,...,0.0,0.0,0.0,0.0,0.0,422.0,0.0,0.0,0.0,0.0


Drop not needed columns and rename

In [63]:
btw = btw[['AGS', 'year', 'post_2013', 'post_2017', 'Land', 'Erststimme', 'Zweitstimme', 'Wahlberechtigte (A)', 'Wähler (B)', 'Ungültig', 'Union', 'SPD', 'FDP', 'Linke', 'GRÜNE', 'Andere']]
btw.rename(columns = {'Wahlberechtigte (A)':'Wahlberechtigte', 'Wähler (B)': 'Wähler', 'GRÜNE': 'Grüne'}, inplace = True)
btw.head(10)

,AGS,year,post_2013,post_2017,Land,Erststimme,Zweitstimme,Wahlberechtigte,Wähler,Ungültig,Union,SPD,FDP,Linke,Grüne,Andere
0,01001000,2002,0,0,01,1.0,NaN,65740,49054,503.0,16418,26309,2038,759.0,2833,194.0
0,01001000,2002,0,0,01,NaN,1.0,65740,49054,501.0,14362,23396,3424,859.0,5556,956.0
1,01001000,2005,0,0,01,1.0,NaN,66970,49002,758.0,16954,24261,1601,2312.0,2729,387.0
1,01001000,2005,0,0,01,NaN,1.0,66970,49002,767.0,13910,21000,4196,3066.0,5028,1035.0
2,01001000,2009,0,0,01,1.0,NaN,69075,46583,1204.0,13859,16552,4253,4697.0,5520,498.0
2,01001000,2009,0,0,01,NaN,1.0,69075,46583,1152.0,11238,12739,6530,5267.0,7323,2334.0
3,01001000,2013,1,0,01,1.0,NaN,70321,46827,589.0,15647,18915,807,3250.0,4533,3086.0
3,01001000,2013,1,0,01,NaN,1.0,70321,46827,562.0,14136,16480,2061,3780.0,5507,4301.0
4,01001000,2017,1,1,01,1.0,NaN,70105,49694,485.0,15140,15175,3421,5316.0,5958,4199.0
4,01001000,2017,1,1,01,NaN,1.0,70105,49694,423.0,13382,12142,5073,5699.0,7416,5559.0


Check summation of votes

In [64]:
parties = ['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere', 'Ungültig']
btw['votes'] = btw[parties].sum(axis=1)
btw['votes'] = btw['votes'].astype(int) #convert to integer
btw['ok'] = btw['Wähler'].equals(btw['votes'])  # check if votes == Wähler
print(btw['ok'].describe())

count     122592
unique         1
top         True
freq      122592
Name: ok, dtype: object


Calculate vote shares in percent

In [65]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    btw.loc[:, party] = (btw[party] / btw['Wähler'])*100
btw.loc[:, 'Wähler'] = (btw['Wähler'] / btw['Wahlberechtigte'])*100
btw.head(10)

,AGS,year,post_2013,post_2017,Land,Erststimme,Zweitstimme,Wahlberechtigte,Wähler,Ungültig,Union,SPD,FDP,Linke,Grüne,Andere,votes,ok
0,01001000,2002,0,0,01,1.0,NaN,65740,74.618193,503.0,33.469238,53.632731,4.154605,1.547274,5.775268,0.395483,49054,True
0,01001000,2002,0,0,01,NaN,1.0,65740,74.618193,501.0,29.277939,47.694378,6.980063,1.751131,11.326293,1.948873,49054,True
1,01001000,2005,0,0,01,1.0,NaN,66970,73.170076,758.0,34.598588,49.510224,3.267214,4.718175,5.569160,0.789764,49002,True
1,01001000,2005,0,0,01,NaN,1.0,66970,73.170076,767.0,28.386596,42.855394,8.562916,6.256887,10.260806,2.112159,49002,True
2,01001000,2009,0,0,01,1.0,NaN,69075,67.438292,1204.0,29.751197,35.532276,9.129940,10.083078,11.849816,1.069060,46583,True
2,01001000,2009,0,0,01,NaN,1.0,69075,67.438292,1152.0,24.124681,27.346886,14.017989,11.306700,15.720327,5.010412,46583,True
3,01001000,2013,1,0,01,1.0,NaN,70321,66.590350,589.0,33.414483,40.393363,1.723365,6.940440,9.680313,6.590215,46827,True
3,01001000,2013,1,0,01,NaN,1.0,70321,66.590350,562.0,30.187712,35.193371,4.401307,8.072266,11.760309,9.184872,46827,True
4,01001000,2017,1,1,01,1.0,NaN,70105,70.885101,485.0,30.466455,30.536886,6.884131,10.697469,11.989375,8.449712,49694,True
4,01001000,2017,1,1,01,NaN,1.0,70105,70.885101,423.0,26.928804,24.433533,10.208476,11.468185,14.923331,11.186461,49694,True


Export dataset

In [66]:
btw = btw[['AGS', 'year', 'post_2013', 'post_2017', 'Land', 'Erststimme', 'Zweitstimme', 'Wahlberechtigte', 'Wähler', 'Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']]
btw = btw.set_index(['AGS', 'year'])
btw.to_csv(f'{path}/data/btw.csv', encoding = 'utf-8-sig')
btw.head()

post_2013  post_2017 Land  Erststimme  Zweitstimme  \
AGS      year                                                       
01001000 2002          0          0   01         1.0          NaN   
         2002          0          0   01         NaN          1.0   
         2005          0          0   01         1.0          NaN   
         2005          0          0   01         NaN          1.0   
         2009          0          0   01         1.0          NaN   

               Wahlberechtigte     Wähler      Union        SPD       FDP  \
AGS      year                                                               
01001000 2002            65740  74.618193  33.469238  53.632731  4.154605   
         2002            65740  74.618193  29.277939  47.694378  6.980063   
         2005            66970  73.170076  34.598588  49.510224  3.267214   
         2005            66970  73.170076  28.386596  42.855394  8.562916   
         2009            69075  67.438292  29.751197  35.532276  9.129940   

                   Linke      Grüne    Andere  
AGS      year                                  
01001000 2002   1.547274   5.775268  0.395483  
         2002   1.751131  11.326293  1.948873  
         2005   4.718175   5.569160  0.789764  
         2005   6.256887  10.260806  2.112159  
         2009  10.083078  11.849816  1.069060